# Preliminary settings

Libraries imported.

In [1]:
import os
import re
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.layers import TextVectorization

from datetime import datetime
seed = int(round(datetime.now().timestamp()))
np.random.seed(seed)
tf.random.set_seed(seed)

Path to data.

In [2]:
dx = "/data/train/rxxch9vw59.2/"

Important constants.

In [ ]:
# Path to the images
IMAGES_PATH = dx+"images"

# Desired image dimensions
IMAGE_SIZE = (299, 299)

# Vocabulary size
VOCAB_SIZE = 5800
# VOCAB_SIZE = len(final_vocab)

# Fixed length allowed for any sequence
SEQ_LENGTH = 15

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
BATCH_SIZE = 32
EPOCHS = 1
AUTOTUNE = tf.data.AUTOTUNE

# Dataset creation

The image files are loaded. Each image is paired with two captions.
The pairs are shuffled and split into 20% test and 80% train set.

In [4]:
def load_captions_data(filename):
    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the text file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """
    with open(filename, encoding="utf8") as caption_file:
        caption_data = json.load(caption_file)
        caption_mapping = {}
        text_data = []

        for item in caption_data:  # Iterate over the list of dictionaries
            img_name = os.path.join(IMAGES_PATH, item['filename'].strip())  # Access 'filename'
            caption_mapping[img_name] = ["<start> " + caption.strip() + " <end>" for caption in item['caption']]  # Access 'caption'
            text_data.extend(caption_mapping[img_name])

        return caption_mapping, text_data


def train_val_split(caption_data, train_size=0.8, shuffle=True):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Training and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())

    # 2. Shuffle if necessary
    if shuffle:
        np.random.shuffle(all_images)

    # 3. Split into training and validation sets
    train_size = int(len(caption_data) * train_size)

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }
    validation_data = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:]
    }

    # 4. Return the splits
    return training_data, validation_data


# Load the dataset
captions_mapping, text_data = load_captions_data(dx + "captions.json")

# Split the dataset into training and validation sets
train_data, valid_data = train_val_split(captions_mapping)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))

Number of training samples:  7323
Number of validation samples:  1831


In [5]:
# Import necessary libraries
from collections import Counter
import json
import matplotlib.pyplot as plt

# Load the captions data from the file
captions_path = dx + "captions.json"
with open(captions_path, 'r', encoding='utf-8') as f:
    captions_data = json.load(f)

# Flatten all captions into a single list
captions = []
for item in captions_data:
    captions.extend(item['caption'])

# Tokenize the captions by splitting on spaces
all_words = ' '.join(captions).split()

# Count the frequency of each word
word_freq = Counter(all_words)

# Visualize the word frequency distribution
plt.figure(figsize=(10,6))
plt.hist(list(word_freq.values()), bins=50)
plt.yscale('log')
plt.title('Word Frequency Distribution')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

# Return the word frequencies
word_freq.most_common(10)  # Show the 10 most common words for reference


[('আছে।', 9264),
 ('একজন', 8775),
 ('একটি', 6006),
 ('পুরুষ', 4564),
 ('মানুষ', 4091),
 ('দাড়িয়ে', 3489),
 ('বসে', 3379),
 ('ও', 2350),
 ('দিয়ে', 2223),
 ('জন', 2201)]

In [6]:
text_data

['<start> তিন জন মেয়ে মানুষ আছে। এক জন দাড়িয়ে আছে আর দুই জন বসে আছে। <end>',
 '<start> একটি হলুদ জামা পায়জামা পরা মহিলা দাড়িয়ে হাতে একটি বেত নিয়ে পিটানোর ভাব দেখাচ্ছে আর ছোট একটি মেয়ে পিছনে ব্যাগ নিয়ে বসে কাঁদছে। <end>',
 '<start> অনেক মেয়ে মানুষ বসে আছে। <end>',
 '<start> একটি নীল জামা পরা মহিলা একটি নীল ল্যাপটপ এর দিকে তাকিয়ে আছে এবং পিছনে  তার দিকে বসে শারি পরে তাকিয়ে আছে অনেকগুলো মহিলা। <end>',
 '<start> অনেক মানুষ একসাথে বসে কাজ করছে। <end>',
 '<start> ২ টি  ছোট ছেলে একজন শার্ট প্যান্ট  দাড়িয়ে চেয়ে আছে আরেকজন বসে গার্মেন্টস এ কাজ করছে নীল  শার্ট পরে তাদের পিছনে অনেকগুলো মহিলা বসে দাড়িয়ে কাজ করছে। <end>',
 '<start> ছয় জন মানুষ দাড়িয়ে আছে। <end>',
 '<start> ৬ জন মানুষ এলোমেলো দাড়িয়ে আছে, তাদের মাঝে ২ জন ছেলে ৪ জন পুরুষ,  তাদের একজন লুঙ্গী পরে দাড়িয়ে আছে। <end>',
 '<start> এক জন মেয়ে মানুষ মাথায় ঘোমটা দিয়ে কাজ করছে। মাটিতে বিভিন্ন রঙের মসলা আছে। <end>',
 '<start> একটি মহিলা হালকা পানির উপরে দাড়িয়ে আছে শারি পরে, মহিলার মুখ ডানদিকে ঘুরানো, পানির রং হলুদ দেখাচ্ছে। <end>',
 '<start> আবছা

Image augmentation is performed and the captions are vectorised.

In [ ]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


strip_chars = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
strip_chars = strip_chars.replace("<", "")
strip_chars = strip_chars.replace(">", "")

# List of words to include in the final vocabulary
final_vocab = list(frequent_words.keys())
print(f"Final Vocabulary Length: {len(final_vocab)}")

# Initialize the TextVectorization layer with the final vocabulary
vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
    oov_token="<unk>"  # Token for words not in the vocabulary
)
vectorization.set_vocabulary(final_vocab)

# Adapting the vectorization layer
vectorization.adapt(text_data)

# Save the vocabulary to a JSON file after training
vocab = vectorization.get_vocabulary()
with open('/results/vocab.json', 'w', encoding='utf-8') as f:
    json.dump(vocab, f, ensure_ascii=False, indent=4)
print(f"Vocabulary saved with {len(vocab)} tokens.")

# Data augmentation for image data
image_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomContrast(0.3),
    ]
)


2024-10-24 03:43:43.567817: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-24 03:43:43.651941: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-24 03:43:43.652518: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-24 03:43:43.653811: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the app

Vocabulary saved with 5793 tokens.


In [ ]:
covered = sum(word_freq[word] for word in final_vocab)
total = sum(word_freq.values())
print(f"Coverage: {covered / total * 100:.2f}%")

A function to expand the dataset such that each image is paired with each caption separately.

In [8]:
def flatten_dataset(caption_mapping):
    """Flattens the dataset by creating separate (image, caption) pairs for each caption of an image.

    Args:
        caption_mapping: Dictionary mapping image names to a list of captions.

    Returns:
        images: List of image paths (repeated for each caption).
        captions: List of corresponding captions (each caption as a string).
    """
    images = []
    captions = []

    # For each image, repeat the image path for each caption and append to the lists
    for img_path, caption_list in caption_mapping.items():
        for caption in caption_list:
            images.append(img_path)
            captions.append(caption)

    return images, captions


The train and test datasets are created using TensorFlow Dataset.

In [9]:
def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def process_input(img_path, caption):
    img = decode_and_resize(img_path)
    
    # Vectorize the caption
    captions_vectorized = vectorization(caption)
    
    return img, captions_vectorized


# Function for dataset creation
def make_dataset(images, captions):
    # Create a dataset from tensor slices
    dataset = tf.data.Dataset.from_tensor_slices((images, captions))
    dataset = dataset.shuffle(BATCH_SIZE * 8)
    
    # Map images and captions to processed inputs
    
    # Process images and vectorize captions
    dataset = dataset.map(lambda img, cap: process_input(img, cap), num_parallel_calls=AUTOTUNE)
    
    for img, cap in dataset.take(1):
        print(f"Image and caption shape: {img.shape, cap.shape} \n")
        
    # Batch the data
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    return dataset


# def make_dataset(images, captions):
#     print([len(i) for i in images[:5]])
#     print([len(i) for i in captions[:5]])

# Flatten the training data
train_images, train_captions = flatten_dataset(train_data)
valid_images, valid_captions = flatten_dataset(valid_data)

# Pass the list of images and the list of corresponding captions
# Create train and validation datasets
print("Training dataset \n")
train_dataset = make_dataset(train_images, train_captions)

print("Validation dataset \n")
valid_dataset = make_dataset(valid_images, valid_captions)

# Print the shapes of the datasets
for img_batch, cap_batch in train_dataset.take(1):
    print(f"Training batch image shape: {img_batch.shape}, caption shape: {cap_batch.shape}")
for img_batch, cap_batch in valid_dataset.take(1):
    print(f"Validation batch image shape: {img_batch.shape}, caption shape: {cap_batch.shape}")

Training dataset 

Image and caption shape: (TensorShape([299, 299, 3]), TensorShape([15])) 

Validation dataset 



2024-10-24 03:43:57.987149: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


Image and caption shape: (TensorShape([299, 299, 3]), TensorShape([15])) 

Training batch image shape: (32, 299, 299, 3), caption shape: (32, 15)


2024-10-24 03:43:59.715578: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-10-24 03:44:00.457439: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


Validation batch image shape: (32, 299, 299, 3), caption shape: (32, 15)


# Transformer model creation

The Transformer uses the EfficientNetB0 CNN.
The Encoder and Decoder block and Positional Embedding layer is seperately created by inheriting the Layer class.
The Transformer model is created by compiling the layers and inheriting the Model class.

In [10]:
def get_cnn_model():
    base_model = efficientnet.EfficientNetB0(
        input_shape=(*IMAGE_SIZE, 3), include_top=False, weights="imagenet",
    )
    # We freeze our feature extractor
    base_model.trainable = False
    base_model_out = base_model.output
    
    # Reduce the sequence length using a pooling operation
    # Usign GlobalAveragePooling2D to reduce the spatial dimensions
    base_model_out = layers.GlobalAveragePooling2D()(base_model_out)
    
    # Optionally, project the output to match the embedding size
    base_model_out = layers.Dense(EMBED_DIM)(base_model_out)
    
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    
    # Print CNN Model Summary
    print("\nCNN Model Summary:")
    cnn_model.summary()
    return cnn_model


class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.0
        )
        self.layernorm_1 = layers.LayerNormalization() # Potential error source
        self.layernorm_2 = layers.LayerNormalization()
        self.dense_1 = layers.Dense(embed_dim, activation="relu")

    def call(self, inputs, training, mask=None):
        # Input shape
        print(f"Encoder Input Shape: {inputs.shape}")

        print(f"Encoder Input Shape before LayerNorm: {inputs.shape}")
        inputs = self.layernorm_1(inputs)
        print(f"Encoder Input Shape after LayerNorm: {inputs.shape}")
        
        inputs = self.dense_1(inputs)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            training=training,
        )
        
        out_1 = self.layernorm_2(inputs + attention_output_1)

        # Output shape
        print(f"Encoder Output Shape: {out_1.shape}")
        return out_1


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.embed_scale = tf.math.sqrt(tf.cast(embed_dim, tf.float32))

    def call(self, inputs):
        print(f"Positional Embedding Input Shape: {inputs.shape}")
        
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_tokens = embedded_tokens * self.embed_scale
        embedded_positions = self.position_embeddings(positions)
        
        # Embeddings shape
        print(f"Positional Embedding Output Shape: {embedded_tokens.shape}")
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_layer_2 = layers.Dense(embed_dim)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.embedding = PositionalEmbedding(
            embed_dim=EMBED_DIM, sequence_length=SEQ_LENGTH, vocab_size=VOCAB_SIZE
        )
        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")

        self.dropout_1 = layers.Dropout(0.3)
        self.dropout_2 = layers.Dropout(0.5)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, training, mask=None):
        print(f"Decoder Input Shape: {inputs.shape}")
        
        inputs = self.embedding(inputs)
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=combined_mask,
            training=training,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            training=training,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.dropout_1(ffn_out, training=training)
        ffn_out = self.ffn_layer_2(ffn_out)

        ffn_out = self.layernorm_3(ffn_out + out_2, training=training)
        ffn_out = self.dropout_2(ffn_out, training=training)
        preds = self.out(ffn_out)
        
        print(f"Decoder Output Shape: {preds.shape}")
        return preds

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


class ImageCaptioningModel(keras.Model):
    def __init__(
        self, cnn_model, encoder, decoder, num_captions_per_image=2, image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        # print(f"Image Embedding Input Shape before passing to Encoder: {img_embed.shape}")
        
        # batch_seq = tf.expand_dims(batch_seq, axis=1)
        print(f"Batch Sequence Input Shape before slicing: {batch_seq.shape}")
        
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1] # Input sequence (without the last token)

        # print(f"Batch Sequence Input Shape before target sequence: {batch_seq_inp.shape}")
        
        batch_seq_true = batch_seq[:, 1:] # Target sequence (without the first token)
        mask = tf.math.not_equal(batch_seq_true, 0)
        
        print(f"Batch Sequence Input Shape: {batch_seq_inp.shape}")
        print(f"Batch Sequence True Shape: {batch_seq_true.shape}")
        
        batch_seq_pred = self.decoder(
            batch_seq_inp, encoder_out, training=training, mask=mask
        )

        print(f"Batch Sequence Predicted Shape: {batch_seq_pred.shape}")
        
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data

        # batch_seq = tf.expand_dims(batch_seq, axis=1)

        # print(f"Training Image Batch Shape before passing to CNN: {batch_img.shape}")
        total_loss = 0
        total_acc = 0
    
        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        print(f"Training Image Batch Shape: {batch_img.shape}")
        print(f"Training Sequence Batch Shape: {batch_seq.shape}")
        
        # 1. Get image embeddings from CNN
        img_embed = self.cnn_model(batch_img)
        print(f"Image Embeddings Shape: {img_embed.shape}")

        # 2. Reshape CNN output to (batch_size, 1, embedding_dim)
        img_embed = tf.expand_dims(img_embed, axis=1)  # It should be (None, 1, 1024)

        print(f"Reshaped Image Embeddings for Encoder: {img_embed.shape}")
        
        # 3. Make sure batch_seq has 3 dimensions
        if batch_seq.shape.ndims == 2:
            # Reshape the sequence to have a third dimension (e.g., 1 caption per image)
            batch_seq = tf.expand_dims(batch_seq, axis=1)
        
        print(f"Updated Sequence Shape: {batch_seq.shape}")

        # 4. Accumulate loss and accuracy for each caption
        with tf.GradientTape() as tape:
            # Loop through each caption (batch_seq should be (batch_size, num_captions, sequence_length))
            num_captions_per_image = batch_seq.shape[1] # Extract the num_captions dimension
            
            for i in range(num_captions_per_image):
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )
                total_loss += loss
                total_acc += acc

            # 5. Compute the mean loss and accuracy
            avg_loss = total_loss / tf.cast(num_captions_per_image, dtype=tf.float32)
            avg_acc = total_acc / tf.cast(num_captions_per_image, dtype=tf.float32)

        # Backpropagation
        # 6. Get the list of all the trainable weights
        train_vars = self.encoder.trainable_variables + self.decoder.trainable_variables
        
        # 7. Get the gradients (from the accumulated loss)
        grads = tape.gradient(avg_loss, train_vars)
    
        # 8. Update the trainable weights
        self.optimizer.apply_gradients(zip(grads, train_vars))
    
        # 9. Update the trackers
        self.loss_tracker.update_state(avg_loss)
        self.acc_tracker.update_state(avg_acc)
    
        # 10. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data

        # batch_seq = tf.expand_dims(batch_seq, axis=1)

        batch_loss = 0
        batch_acc = 0

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker, self.acc_tracker]


cnn_model = get_cnn_model()
encoder = TransformerEncoderBlock(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=1)
decoder = TransformerDecoderBlock(embed_dim=EMBED_DIM, ff_dim=FF_DIM, num_heads=2)
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=image_augmentation,
)

16719872/16705208 [==============================] - 3s 0us/step

CNN Model Summary:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 299, 299, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 301, 301, 3)  0           normalization[0][0]              
_________

The loss function and early stopping is defined. The model is compiled with the same.

In [11]:
# Define the loss function
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)

# EarlyStopping criteria
early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)


# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,
        )


# Create a learning rate schedule
num_train_steps = len(train_dataset) * EPOCHS
num_warmup_steps = num_train_steps // 15
lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# Compile the model
caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)

# Model training and testing

Sets the version and checks if the model has been previously trained or being trained for the first time. If saved weights are found, they're loaded.

In [12]:
mdx = '231005'  # Sets the version
tmpx = f'/results/Model_weights/{mdx}/Temp/'

# Check if the directory exists
if not os.path.exists(tmpx):
    print(f"Directory {tmpx} does not exist. Creating the directory.")
    os.makedirs(tmpx)  # Create the directory if it doesn't exist

# Now check for the files
try:
    weight_path = f'{tmpx}imgcap_{mdx}'
    fls = os.listdir(tmpx)

    # Look for specific weight files (like .index or .data-00000-of-00001)
    checkpoint_files = [f for f in fls if "imgcap_" in f]
    
    if len(checkpoint_files) > 0:
        print("Found saved weights, loading them now...")
        caption_model.load_weights(weight_path)
        print("Saved weights loaded successfully")
    else:
        print("No saved weights found, training from scratch")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Found saved weights, loading them now...
An unexpected error occurred: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /results/Model_weights/231005/Temp/imgcap_231005


Lookup dictionary created and output sequence length is set.

In [13]:
# Load the vocabulary during inference
with open('/results/vocab.json', 'r', encoding='utf-8') as f:
    vocab = json.load(f)

# Recreate the vectorization object and set its vocabulary
vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
)

# Set the vocabulary during inference
vectorization.set_vocabulary(vocab)

print(f"Vocabulary loaded with {len(vocab)} tokens.")

index_lookup = dict(zip(range(len(vocab)), vocab))
max_decoded_sentence_length = SEQ_LENGTH - 1
valid_images = list(valid_data.keys())

Vocabulary loaded with 5793 tokens.


The function reads an image from the given path. It uses the image to generate an caption. The same are displayed.

In [14]:
# Handle invalid token indices
def generate_caption(dt, ix):
    # Select a random image from the validation dataset
    sample_img = f'{dt}images/{ix}'

    # Read the image from the disk
    sample_img = decode_and_resize(sample_img)
    img = sample_img.numpy().clip(0, 255).astype(np.uint8)
    plt.axis('off')
    plt.imshow(img)
    plt.show()
    
    # Pass the image to the CNN
    img = tf.expand_dims(sample_img, 0)
    img = caption_model.cnn_model(img)

    # Expand dimensions to make it compatible with the encoder
    img = tf.expand_dims(img, 1)  # Adding sequence dimension, shape becomes (batch_size, 1, embed_dim)

    # Pass the image features to the Transformer encoder
    encoded_img = caption_model.encoder(img, training=False)

    # Generate the caption using the Transformer decoder
    decoded_caption = "<start> "
    for i in range(max_decoded_sentence_length):
        # Ensure decoded_caption is passed as a list of strings
        tokenized_caption = vectorization(tf.constant([decoded_caption]))[:, :-1]
        
        # Create mask for the tokenized caption
        mask = tf.math.not_equal(tokenized_caption, 0)

        predictions = caption_model.decoder(
            tokenized_caption, encoded_img, training=False, mask=mask
        )

        # Get the predicted token
        sampled_token_index = np.argmax(predictions[0, i, :])

        # Check if sampled_token_index is in the vocabulary range
        if sampled_token_index >= len(vocab):
            print(f"Warning: Token index {sampled_token_index} out of range")
            continue  # Skip to the next iteration if the token is out of range

        sampled_token = index_lookup[sampled_token_index]

        if sampled_token == "<end>":
            break

        decoded_caption += " " + sampled_token

    # Clean up the decoded caption
    decoded_caption = decoded_caption.replace("<start> ", "")
    decoded_caption = decoded_caption.replace(" <end>", "").strip()
    print("\nPredicted Caption: ", decoded_caption)
    print()


Test image path.

In [15]:
img_dt = "/data/test/rxxch9vw59-2/"
imgs = os.listdir(img_dt+"images")

Generates a caption for a random image among test images.

In [15]:
random_image = np.random.choice(imgs)

'1133.png'

In [16]:
# generate_caption(img_dt,random_image)
generate_caption(img_dt,'1228.png')

/tmp/ipykernel_36/3511622986.py:10: UserWarning: Attempted to set non-positive top ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.imshow(img)
2024-10-24 03:45:37.007822: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401
2024-10-24 03:45:38.364302: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-24 03:45:38.365319: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-24 03:45:38.365361: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-10-24 03:45:38.365860: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-24 03:45:38.365960: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx

Encoder Input Shape: (1, 1, 512)
Encoder Input Shape before LayerNorm: (1, 1, 512)
Encoder Input Shape after LayerNorm: (1, 1, 512)
Encoder Output Shape: (1, 1, 512)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Sha

The model is trained.

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath=f'{tmpx}imgcap_{mdx}', 
    save_weights_only=True, 
    save_best_only=True
)

try:
    # Fit the model
    caption_model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=valid_dataset,
        callbacks=[early_stopping, checkpoint_cb],
    )
except ValueError as e:
    print(f"ValueError during model fitting: {e}")

Training Image Batch Shape: (None, 299, 299, 3)
Training Sequence Batch Shape: (None, None)
Image Embeddings Shape: (None, 512)
Reshaped Image Embeddings for Encoder: (None, 1, 512)
Updated Sequence Shape: (None, 1, None)
Batch Sequence Input Shape before slicing: (None, None)
Encoder Input Shape: (None, 1, 512)
Encoder Input Shape before LayerNorm: (None, 1, 512)
Encoder Input Shape after LayerNorm: (None, 1, 512)
Encoder Output Shape: (None, 1, 512)
Batch Sequence Input Shape: (None, None)
Batch Sequence True Shape: (None, None)
Decoder Input Shape: (None, None)
Positional Embedding Input Shape: (None, None)
Positional Embedding Output Shape: (None, None, 512)
Decoder Output Shape: (None, None, 5800)
Batch Sequence Predicted Shape: (None, None, 5800)
Training Image Batch Shape: (None, 299, 299, 3)
Training Sequence Batch Shape: (None, None)
Image Embeddings Shape: (None, 512)
Reshaped Image Embeddings for Encoder: (None, 1, 512)
Updated Sequence Shape: (None, 1, None)
Batch Sequence 

2024-10-24 03:46:13.962748: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-10-24 03:46:15.097128: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2024-10-24 03:46:15.203416: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-10-24 03:46:15.437039: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-10-24 03:46:15.535201: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.01GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2024-10-24 03:46:15.873948: W tensorflow/cor

  2/458 [..............................] - ETA: 3:53 - loss: 8.8352 - acc: 0.0000e+00 

2024-10-24 03:46:17.372616: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2024-10-24 03:46:17.402137: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


  3/458 [..............................] - ETA: 3:52 - loss: 8.8443 - acc: 0.0000e+00

...

443/458 [============================>.] - ETA: 8s - loss: 5.1196 - acc: 0.1585

2024-10-24 03:50:31.513190: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


457/458 [============================>.] - ETA: 0s - loss: 5.0930 - acc: 0.1607

2024-10-24 03:50:42.711136: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


458/458 [==============================] - ETA: 0s - loss: 5.0902 - acc: 0.1608ValueError during model fitting: in user code:

    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1330, in test_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1320, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1313, in run_step  **
        outputs = model.test_step(data)
    File "/tmp/ipykernel_36/364036858.py", line 308, in test_step
        img_embed, batch_seq[:, i, :], training=False

    ValueError: Index out of range using input dim 2; input has only 2 dims for '{{node strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_INT64, begin_mask=5, ellipsis_mask=0, end_mask=5, new_axis_mask=0, shrink_axis_mask=2](IteratorGetNext:1, strided_slice/stack, strided_sl

Generate captions for all test images.

In [18]:
for i in imgs:
  generate_caption(img_dt, i)

/tmp/ipykernel_36/3511622986.py:10: UserWarning: Attempted to set non-positive top ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.imshow(img)


Encoder Input Shape: (1, 1, 512)
Encoder Input Shape before LayerNorm: (1, 1, 512)
Encoder Input Shape after LayerNorm: (1, 1, 512)
Encoder Output Shape: (1, 1, 512)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)

Predicted Caption:  একজন পুরুষ দাড়িয়ে আছে।

Encoder Input 

2024-10-24 03:53:03.844806: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


Encoder Input Shape: (1, 1, 512)
Encoder Input Shape before LayerNorm: (1, 1, 512)
Encoder Input Shape after LayerNorm: (1, 1, 512)
Encoder Output Shape: (1, 1, 512)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)

Predicted Caption:  একজন পুরুষ আছে।



2024-10-24 03:53:04.077856: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


...

Encoder Input Shape: (1, 1, 512)
Encoder Input Shape before LayerNorm: (1, 1, 512)
Encoder Input Shape after LayerNorm: (1, 1, 512)
Encoder Output Shape: (1, 1, 512)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)

Predicted Caption:  কয়েকজন মানুষ আছে।

Encoder Input Shape: (1, 1, 512)
Encoder Input Shape before LayerNorm: (1, 1, 512)
Encoder Input Shape after LayerNorm: (1, 1, 512)
Encoder Output Shape: (1, 1, 512)
Decoder 

2024-10-24 03:59:04.514125: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


Encoder Input Shape: (1, 1, 512)
Encoder Input Shape before LayerNorm: (1, 1, 512)
Encoder Input Shape after LayerNorm: (1, 1, 512)
Encoder Output Shape: (1, 1, 512)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)

Predicted Caption:  একজন পুরুষ দাড়িয়ে আছে।

Encoder Input 

2024-10-24 03:59:29.178725: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


Encoder Input Shape: (1, 1, 512)
Encoder Input Shape before LayerNorm: (1, 1, 512)
Encoder Input Shape after LayerNorm: (1, 1, 512)
Encoder Output Shape: (1, 1, 512)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)
Decoder Input Shape: (1, 14)
Positional Embedding Input Shape: (1, 14)
Positional Embedding Output Shape: (1, 14, 512)
Decoder Output Shape: (1, 14, 5800)

Predicted Caption:  একজন পুরুষ দাড়িয়ে আছে।

Encoder Input 

Save the weights of the trained model.

In [21]:
# After training the model, save the weights
caption_model.save_weights(f'{tmpx}imgcap_{mdx}', save_format='tf')

Dump the vectorised vocabulary.

In [22]:
# Define the directory path
directory = f'/results/Vocab/{mdx}'

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Save the vocabulary using pickle
with open(f'{directory}/vocab_{mdx}', 'wb') as f:
    pickle.dump(vocab, f)